In [1]:
import numpy as np
from numba import jit

alpha=0.36 # capital share in production function
beta=0.97 # discount rate
sigma=1 # CRRA parameter
delta=1 # depreciation rate
k_ss=((alpha*beta)/(beta*delta-beta+1))**(1/(1-alpha)) # steady-state capital
k_min=0.8*k_ss # lowest k
k_max=1.2*k_ss # highest k
grid=200 # numbers of grid points
K=np.linspace(k_min, k_max, grid) # interpolate capital uniformly to the grid points

kappa=0.5 # Markov transition probability
epsilon=0.05 # size of tech shocks
P=[0.5, 0.5] # stationary distribution
Z=[-epsilon, epsilon] # shocks
pi=np.array([[kappa, 1-kappa], [1-kappa, kappa]]) # transition matrix

max_iter=500 # maximum of iteration
tol=1e-4 # tolerance

@jit
def u(c, sigma):
    # CRRA utility function
    if sigma==1:
        return np.log(c)
    else:
        return (c**(1-sigma))/(1-sigma)
    
@jit
def bellman(v):
    """
    bellman operator which transforms initial function v to Tv according to:
    
    Tv(k, z)=\max _{k'} u(e^z k**\alpha +(1-\delta )k-k')+beta* \Sigma _{z'\in Z} v(k', z')\pi(z'\vert z)
    
    subject to:
    0\leq k' \leq e^z k**\alpha +(1-\delta )k
    and 
    k' \in K
    ------
    parameters:
    v(k, z): initial value function
    ndarray, 2 \times grid
    
    output:
    Tv(k, z): updated value function
    ndarray, 2 \times grid
    k'(k, z): control variable associated with Tv
    ndarray, 2 \times grid
    """
    # first, calculate the pseudo_v function:
    # \tilde v(k, z, k')=\u(e^z k**\alpha +(1-\delta )k-k')+beta* \Sigma _{z'\in Z} v(k', z')\pi(z'\vert z)
    #with the constraint: 
    # 0\leq k' \leq e^z k**\alpha +(1-\delta )k
    pseudo_v=np.zeros([2, grid, grid])
    for s in range(2):
        for i in range(grid):
            k_prime_max=np.exp(Z[s]) * K[i]**alpha + (1-delta)*K[i]
            for j in range(grid):
                if K[j]<k_prime_max:
                    pseudo_v[s, i, j]=(u(np.exp(Z[s]) * K[i]**alpha + (1-delta)*K[i]-K[j], sigma) +
                    beta*(v[s, j]*pi[s, s]+v[1-s, j]*pi[s, 1-s]))
                else:
                    pseudo_v[s, i, j]=pseudo_v[s, i, j-1]
    # next, maximize pseudo_v row by row to get Tv(k) and k'(k)
    new_v=np.zeros([2, grid])
    K_prime=np.zeros([2, grid])
    for s in range(2):
        K_prime_index=np.argmax(pseudo_v[s], axis=1)
        for i in range(grid):
            new_v[s, i]=max(pseudo_v[s, i])
            K_prime[s, i]=K[K_prime_index[i]]
    return new_v, K_prime

def vfi(v):
    """
    value function iteration until convergence or max_iter attained
    ------
    output:
    n: numbers of iteration
    integer
    V[n]: optimal value function
    ndarray, 2 \times grid
    K_p[n-1]: optimal policy function
    ndarray, 2 \times grid
    """
    V_before=v
    n=1
    while n<=max_iter+1:
        V_after, K_p=bellman(V_before)
        if max(max(abs(V_before[0]-V_after[0])), max(abs(V_before[1]-V_after[1])))<=tol:
            return n, V_after, K_p
            break
        else:
            V_before=V_after
            n=n+1

# saving the optimal results
v_0=-30*np.ones([2, grid])
n_iter, V_opt, K_prime_opt=vfi(v_0)

In [2]:
alpha=0.36 # capital share in production function
beta=0.97 # discount rate
sigma=2 # CRRA parameter
delta=0.06 # depreciation rate
k_ss=((alpha*beta)/(beta*delta-beta+1))**(1/(1-alpha)) # steady-state capital
k_min=0.8*k_ss # lowest k
k_max=1.2*k_ss # highest k
grid=200 # numbers of grid points
K=np.linspace(k_min, k_max, grid) # interpolate capital uniformly to the grid points

kappa=0.975 # Markov transition probability
epsilon=0.022 # size of tech shocks
P=[0.5, 0.5] # stationary distribution
Z=[-epsilon, epsilon] # shocks
pi=np.array([[kappa, 1-kappa], [1-kappa, kappa]]) # transition matrix

max_iter=500 # maximum of iteration
tol=1e-4 # tolerance

@jit
def u(c, sigma):
    # CRRA utility function
    if sigma==1:
        return np.log(c)
    else:
        return (c**(1-sigma))/(1-sigma)
    
@jit
def bellman(v):
    """
    bellman operator which transforms initial function v to Tv according to:
    
    Tv(k, z)=\max _{k'} u(e^z k**\alpha +(1-\delta )k-k')+beta* \Sigma _{z'\in Z} v(k', z')\pi(z'\vert z)
    
    subject to:
    0\leq k' \leq e^z k**\alpha +(1-\delta )k
    and 
    k' \in K
    ------
    parameters:
    v(k, z): initial value function
    ndarray, 2 \times grid
    
    output:
    Tv(k, z): updated value function
    ndarray, 2 \times grid
    k'(k, z): control variable associated with Tv
    ndarray, 2 \times grid
    """
    # first, calculate the pseudo_v function:
    # \tilde v(k, z, k')=\u(e^z k**\alpha +(1-\delta )k-k')+beta* \Sigma _{z'\in Z} v(k', z')\pi(z'\vert z)
    #with the constraint: 
    # 0\leq k' \leq e^z k**\alpha +(1-\delta )k
    pseudo_v=np.zeros([2, grid, grid])
    for s in range(2):
        for i in range(grid):
            k_prime_max=np.exp(Z[s]) * K[i]**alpha + (1-delta)*K[i]
            for j in range(grid):
                if K[j]<k_prime_max:
                    pseudo_v[s, i, j]=(u(np.exp(Z[s]) * K[i]**alpha + (1-delta)*K[i]-K[j], sigma) +
                    beta*(v[s, j]*pi[s, s]+v[1-s, j]*pi[s, 1-s]))
                else:
                    pseudo_v[s, i, j]=pseudo_v[s, i, j-1]
    # next, maximize pseudo_v row by row to get Tv(k) and k'(k)
    new_v=np.zeros([2, grid])
    K_prime=np.zeros([2, grid])
    for s in range(2):
        K_prime_index=np.argmax(pseudo_v[s], axis=1)
        for i in range(grid):
            new_v[s, i]=max(pseudo_v[s, i])
            K_prime[s, i]=K[K_prime_index[i]]
    return new_v, K_prime

def vfi(v):
    """
    value function iteration until convergence or max_iter attained
    ------
    output:
    n: numbers of iteration
    integer
    V[n]: optimal value function
    ndarray, 2 \times grid
    K_p[n-1]: optimal policy function
    ndarray, 2 \times grid
    """
    V_before=v
    n=1
    while n<=max_iter+1:
        V_after, K_p=bellman(V_before)
        if max(max(abs(V_before[0]-V_after[0])), max(abs(V_before[1]-V_after[1])))<=tol:
            return n, V_after, K_p
            break
        else:
            V_before=V_after
            n=n+1

# saving the optimal results
v_0=-30*np.ones([2, grid])
n_iter1, V_opt1, K_prime_opt1=vfi(v_0)

In [3]:
alpha=0.36 # capital share in production function
beta=0.97 # discount rate
sigma=1 # CRRA parameter
delta=1 # depreciation rate
k_ss=((alpha*beta)/(beta*delta-beta+1))**(1/(1-alpha)) # steady-state capital
k_min=0.8*k_ss # lowest k
k_max=1.2*k_ss # highest k
grid=200 # numbers of grid points
K=np.linspace(k_min, k_max, grid) # interpolate capital uniformly to the grid points

In [4]:
alpha=0.36 # capital share in production function
beta=0.97 # discount rate
sigma=2 # CRRA parameter
delta=0.06 # depreciation rate
k_ss=((alpha*beta)/(beta*delta-beta+1))**(1/(1-alpha)) # steady-state capital
k_min=0.8*k_ss # lowest k
k_max=1.2*k_ss # highest k
grid=200 # numbers of grid points
K1=np.linspace(k_min, k_max, grid) # interpolate capital uniformly to the grid points

In [ ]:
import matplotlib.pyplot as plt

fig, ax=plt.subplots(2, 2, figsize=(7, 6))

ax[0, 0].plot(K, V_opt[0], label=r'$z_1=-\epsilon$')
ax[0, 0].plot(K, V_opt[1], label=r'$z_2=+\epsilon$', linestyle=':')
ax[0, 0].grid()
ax[0, 0].set_xlabel(r'$k$', fontsize=11)
ax[0, 0].set_ylabel(r'$\nu (k)$', fontsize=11)
ax[0, 0].legend(loc='lower right', fontsize=11)
ax[0, 0].spines['right'].set_visible(False)
ax[0, 0].spines['top'].set_visible(False)
ax[0, 0].set_title('model 1: value functions', fontsize=12.5, position=[.5, 1.02])
ax[0, 0].patch.set_facecolor('grey')
ax[0, 0].patch.set_alpha(0.06)

ax[0, 1].plot(K, K_prime_opt[0], label=r'$z_1=-\epsilon$')
ax[0, 1].plot(K, K_prime_opt[1], label=r'$z_2=+\epsilon$', linestyle=':')
ax[0, 1].grid()
ax[0, 1].set_xlabel(r'$k$', fontsize=11)
ax[0, 1].set_ylabel(r'$k^{\prime} (k)$', fontsize=11)
ax[0, 1].legend(loc='lower right', fontsize=11)
ax[0, 1].spines['right'].set_visible(False)
ax[0, 1].spines['top'].set_visible(False)
ax[0, 1].set_title('model 1: policy functions', fontsize=12.5, position=[.5, 1.02])
ax[0, 1].patch.set_facecolor('grey')
ax[0, 1].patch.set_alpha(0.06)

ax[1, 0].plot(K1, V_opt1[0], label=r'$z_1=-\epsilon$')
ax[1, 0].plot(K1, V_opt1[1], label=r'$z_2=+\epsilon$', linestyle=':')
ax[1, 0].grid()
ax[1, 0].set_xlabel(r'$k$', fontsize=11)
ax[1, 0].set_ylabel(r'$\nu (k)$', fontsize=11)
ax[1, 0].legend(loc='lower right', fontsize=11)
ax[1, 0].spines['right'].set_visible(False)
ax[1, 0].spines['top'].set_visible(False)
ax[1, 0].set_title('model 2: value functions', fontsize=12.5, position=[.5, 1.02])
ax[1, 0].patch.set_facecolor('grey')
ax[1, 0].patch.set_alpha(0.06)

ax[1, 1].plot(K1, K_prime_opt1[0], label=r'$z_1=-\epsilon$')
ax[1, 1].plot(K1, K_prime_opt1[1], label=r'$z_2=+\epsilon$', linestyle=':')
ax[1, 1].grid()
ax[1, 1].set_xlabel(r'$k$', fontsize=11)
ax[1, 1].set_ylabel(r'$k^{\prime} (k)$', fontsize=11)
ax[1, 1].legend(loc='lower right', fontsize=11)
ax[1, 1].spines['right'].set_visible(False)
ax[1, 1].spines['top'].set_visible(False)
ax[1, 1].set_title('model 2: policy functions', fontsize=12.5, position=[.5, 1.02])
ax[1, 1].patch.set_facecolor('grey')
ax[1, 1].patch.set_alpha(0.06)

plt.tight_layout()
plt.show()
#plt.savefig('macro_hw2.pdf', dpi=300)